In [1]:
import subprocess
import os
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
from folium import plugins
import branca.colormap as cm

In [2]:
lat = widgets.FloatText(
    value=34.05,
    description='Lat',
)
lon = widgets.FloatText(
    value=-118.2,
    description='Lon',
)
width = widgets.FloatText(
    value=2,
    description='Width',
)
height = widgets.FloatText(
    value=2,
    description='Height')
t1 = widgets.DatePicker(description='Time 1')
t2 = widgets.DatePicker(description='Time 2')
all_widgets = [lat, lon, width, height, t1, t2]
widgets.GridBox(all_widgets, layout=widgets.Layout(
    grid_template_columns='repeat(2,400px)'))

GridBox(children=(FloatText(value=34.05, description='Lat'), FloatText(value=-118.2, description='Lon'), Float…

In [12]:
disp_args = f'python getDisplacement.py --lat {lat.value} --lon {lon.value} --width {width.value} --height {height.value} -t1 {t1.value.strftime("%Y-%m-%d")} -t2 {t2.value.strftime("%Y-%m-%d")} -o displacement'

In [13]:
process = subprocess.getoutput(disp_args)

In [14]:
process

''

In [32]:
def load_gps_data(filename):
    '''
    

    Parameters
    ----------
    filename : 
        GPS data as retreived from getDisplacement.py
    -------
    Pandas Dataframe of Lon, Lat, and Delta Values

    '''
    gps_in = np.loadtxt(filename, skiprows=(1), usecols=(1,2,3,4,5,6,7,8))
    gps_dict = {'Lon': gps_in[:,0], 'Lat': gps_in[:,1], 'Delta E': gps_in[:,2],
                'Delta N': gps_in[:,3], 'Delta V': gps_in[:,4], 'Sigma E': gps_in[:,5], 'Sigma N':gps_in[:,6], 'Sigma V': gps_in[:,7]}
    gps_df = pd.DataFrame(gps_dict)
    sites= np.loadtxt('displacement_table.txt', skiprows=(1), usecols=(0),dtype=str)
    gps_df['Site'] = sites
    
    return gps_df

In [33]:
gps_disp = load_gps_data('displacement_table.txt')

In [36]:
gps_disp

,Lon,Lat,Delta E,Delta N,Delta V,Sigma E,Sigma N,Sigma V,Site
0,-117.444894,33.274266,-11.623457,7.555908,1.757204,0.536153,0.336497,1.189939,ACSB
1,-117.476508,33.381136,-11.518248,7.886433,0.284070,0.532028,0.333797,1.171877,AR53
2,-117.896492,34.126020,1.200919,2.551658,6.920052,0.546998,0.346955,1.309424,AZU1
3,-119.029741,33.480454,-11.838308,9.434502,-0.231097,0.558917,0.359545,1.421532,BAR1
4,-118.094704,33.962260,-12.227482,8.541456,-3.698405,0.560402,0.370908,1.341388,BKMS
...,...,...,...,...,...,...,...,...,...
141,-118.983692,35.010849,-10.331163,2.469084,-2.310315,0.559997,0.387760,1.386020,WGPP
142,-117.643462,33.674492,-11.857708,8.863337,-2.090861,0.705701,0.408285,1.358633,WHYT
143,-118.741522,34.568500,-10.720889,6.712830,2.139161,0.526235,0.337018,1.189340,WKPK
144,-118.414296,34.259426,-12.770064,6.377454,1.856510,0.725214,0.453373,1.649495,WMAP


In [87]:
def create_map(df, lon_col='Lon', lat_col='Lat', data_col='Delta N',site_col='Site'):
    '''
    Parameters
    ----------
    df :
        Pandas dataframe with column of lat, long, and a data column
    lon_col:
        Name of column with longitude coords. Default value of 'Lon'
    lat_col:
        Name of column with lattitude coords. Default value of 'Lat'
    data_col:
        Name of column with values to be colormapped. Default is 'Delta N'
    -------
    Returns:
        Folium map object
    '''

    start_cords = (df[lat_col].median(),
                 df[lon_col].median())
    my_map = folium.Map(start_cords,
                        tiles = 'Stamen Terrain',
                        zoom_start = 8, width = 600, height = 500,
                       prefer_canvas =True)
    plugins.Fullscreen(position='topleft').add_to(my_map)
    
    # making MPL seismic colormap in Branca
    colors_list = ['blue', 'white', 'red']
    minimum = df[data_col].min()
    maximum = df[data_col].max()
    colormap = cm.LinearColormap(colors = colors_list
                        ,vmin = minimum
                        ,vmax= maximum)
    stations = folium.FeatureGroup(name=f'Displacement for {df[data_col].name}')
    for cord, disp, site in zip(zip(df[lat_col], df[lon_col]),df[data_col], df[site_col]):
        folium.Circle(
            location=cord,
            radius=50.0,
            fill=True,
            color=colormap(disp),
            popup=f'Disp:{disp}    Site:{site}',
            ).add_to(stations)
    my_map.add_child(stations)
    my_map.add_child(colormap)
    my_map.add_child(folium.LayerControl(postion='bottomleft'))
    return my_map

In [88]:
create_map(gps_disp)

In [52]:
gps_disp['Lon'].median()

-118.0769835